<a href="https://colab.research.google.com/github/aditya2k5/women_corelation_dataset/blob/main/wemon_corelation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

women co-relation


In [1]:
print("hello world")

hello world
